In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "martial-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"]
data = pd.read_csv('/Users/swithana/git/icicle_model_card/tests/data/adult/adult.data', names=columns)

In [3]:
data.head()

,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pre-processing

In [4]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

# Convert categorical variables to one-hot encoding
data = pd.get_dummies(data, drop_first=True, dtype=float)

data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = data.drop('income', axis=1).values
y = data['income'].values

print(data.columns.tolist())

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'martial-status_ Married-AF-spouse', 'martial-status_ Married-civ-spouse', 'martial-status_ Married-spouse-absent', 'martial-status_ Never-married', 'martial-status_ Separated', 'martial-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupa

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
733/733 [==============================] - 1s 705us/step - loss: 154.4950 - accuracy: 0.6728 - val_loss: 16.0360 - val_accuracy: 0.7965
Epoch 2/100
733/733 [==============================] - 0s 637us/step - loss: 89.8579 - accuracy: 0.6788 - val_loss: 360.2345 - val_accuracy: 0.2365
Epoch 3/100
733/733 [==============================] - 0s 626us/step - loss: 89.7380 - accuracy: 0.6807 - val_loss: 149.2597 - val_accuracy: 0.7877
Epoch 4/100
733/733 [==============================] - 0s 619us/step - loss: 71.6848 - accuracy: 0.6847 - val_loss: 32.8817 - val_accuracy: 0.8050
Epoch 5/100
733/733 [==============================] - 0s 625us/step - loss: 69.2892 - accuracy: 0.6815 - val_loss: 93.8003 - val_accuracy: 0.7996
Epoch 6/100
733/733 [==============================] - 0s 614us/step - loss: 90.7568 - accuracy: 0.6793 - val_loss: 25.0812 - val_accuracy: 0.7931
Epoch 7/100
733/733 [==============================] - 0s 629us/step - loss: 71.2651 - accuracy: 0.6814 - val_loss:

In [9]:
loss, accuracy = model.evaluate(X_test, y_test)

204/204 [==============================] - 0s 383us/step - loss: 2.4374 - accuracy: 0.3828


In [10]:
y_pred = model.predict(X_test)

204/204 [==============================] - 0s 345us/step


## Model Card Generation

In [11]:
from icicle_model_card.icicle_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis, validate_mc, Metric
import json

In [12]:
mc = ModelCard(
            name="UCI Adult Data Analysis",
            version="0.1",
            short_description="UCI Adult Data analysis using Tensorflow",
            full_description="Using a tensorflow trained neural network to analyse fairness and explainability in the UCI Adult Dataset",
            keywords="uci adult, tensorflow, explainability, fairness, fairlearn, shap",
            author="Sachith Withana"
        )

mc.input_data = 'https://archive.ics.uci.edu/dataset/2/adult'

In [13]:
model_metrics = []
model_metrics.append(Metric("Test loss", loss))
model_metrics.append(Metric("Test Accuracy", accuracy))

ai_model = AIModel(
            name="UCI Adult tensorflow model",
            version="0.1",
            description="Census classification problem using Neural Network",
            owner="Sachith Withana",
            location="github.com/swsachith/ai-model",
            licence="BSD-3 Clause",
            model_structure = json.dumps(json.loads(model.to_json()), indent=2)
        )
ai_model.metrics = model_metrics
mc.ai_model = ai_model

In [14]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()
mc.populate_bias(X_test, y_test, y_pred, "gender", X_test[:, 58], model)

204/204 [==============================] - 0s 313us/step


/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/swithana/git/jupyter/venv/lib/python3.9/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) e

In [15]:
print(mc.bias_analysis)

{'demographic_parity_diff': 0.1290871365646512, 'equal_odds_difference': 0.11184237312832124, 'true_positives': 1560, 'true_negatives': 933, 'false_positives': 4009, 'false_negatives': 11}


In [16]:
print(mc)

{
  "name": "UCI Adult Data Analysis",
  "version": "0.1",
  "short_description": "UCI Adult Data analysis using Tensorflow",
  "full_description": "Using a tensorflow trained neural network to analyse fairness and explainability in the UCI Adult Dataset",
  "keywords": "uci adult, tensorflow, explainability, fairness, fairlearn, shap",
  "author": "Sachith Withana",
  "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
  "output_data": "",
  "ai_model": {
    "name": "UCI Adult tensorflow model",
    "version": "0.1",
    "description": "Census classification problem using Neural Network",
    "owner": "Sachith Withana",
    "location": "github.com/swsachith/ai-model",
    "licence": "BSD-3 Clause",
    "model_structure": "{\n  \"class_name\": \"Sequential\",\n  \"config\": {\n    \"name\": \"sequential\",\n    \"layers\": [\n      {\n        \"module\": \"keras.layers\",\n        \"class_name\": \"InputLayer\",\n        \"config\": {\n          \"batch_input_shape\": [\n    